# 데이터 로드 

In [3]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
from tqdm import tqdm 
from glob import glob 
import os 
from typing import Sequence, Tuple

In [ ]:
!tar -xvf '/content/drive/MyDrive/데이터보관/공유용/train_features.tar'

In [149]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
csvs_dir = glob('/content/katl/*.bz2')
csvs_dir.sort()
csvs_dir = pd.DataFrame(csvs_dir,columns=['dir'])
csvs_name = list(csvs_dir['dir'].apply(lambda x : x.split('katl/')[-1].split('.csv')[0].split('_',maxsplit=1)[-1]))

In [5]:
def make_dir(dir,df):
  csv_dir = df[df['dir'].apply(lambda x : dir in x )].values[0][0]
  return csv_dir 

In [ ]:
# for csv_name in tqdm(csvs_name):
#   exec(f"{csv_name} = pd.read_csv(make_dir('{csv_name}',csvs_dir))")

In [8]:
airport_config_df = pd.read_csv(make_dir('config',csvs_dir))

# 데이터 전처리 

## 30분 단위 분할 

In [9]:
airport_config_df['timestamp'] = pd.to_datetime(airport_config_df['timestamp'])
airport_config = airport_config_df.sort_values(by='timestamp')

In [10]:
airport_config = pd.DataFrame(airport_config.set_index('timestamp').airport_config.resample('30min').ffill().dropna()).reset_index()

In [11]:
airport_config

,timestamp,airport_config
0,2020-11-01 01:30:00,D_8R_9L_A_10_8L_9R
1,2020-11-01 02:00:00,D_8R_9L_A_10_8L_9R
2,2020-11-01 02:30:00,D_8R_9L_A_10_8L_9R
3,2020-11-01 03:00:00,D_8R_9L_A_10_8L_9R
4,2020-11-01 03:30:00,D_8R_9L_A_10_8L_9R
...,...,...
17512,2021-10-31 21:30:00,D_26L_27R_A_26R_27L_28
17513,2021-10-31 22:00:00,D_26L_27R_A_26R_27L_28
17514,2021-10-31 22:30:00,D_26L_27R_A_26R_27L_28
17515,2021-10-31 23:00:00,D_26L_27R_A_26R_27L_28


## One hot Encoding 

In [12]:
open_train_label = pd.read_csv('/content/drive/MyDrive/데이터보관/공유용/open_train_labels (1).csv.bz2')

In [13]:
labels = pd.get_dummies(open_train_label[open_train_label['airport']=='katl'],columns = ['config']).columns[4:]
labels = list(pd.Series(labels).apply(lambda x : x.split(':')[-1]))

In [14]:
for label in labels:
  airport_config[label]='-'

In [15]:
def one_hot(config,label):
  if config == label:
    return 1 
  else:
    return 0

In [16]:
for label in labels:
  airport_config[label] = airport_config['airport_config'].apply(lambda x: one_hot(x,label))

In [17]:
airport_config

,timestamp,airport_config,D_10_8L_A_10_8L,D_10_8R_9L_A_10_8L_9R,D_10_8R_A_10_8R,D_26L_27L_A_26R_27L_28,D_26L_27R_28_A_26R_27L_28,D_26L_27R_A_26L_27L_28,D_26L_27R_A_26L_27R,D_26L_27R_A_26L_27R_28,...,D_26R_28_A_26R_28,D_8L_9L_A_10_8L_9R,D_8R_9L_A_10_8L_9R,D_8R_9L_A_10_8R_9R,D_8R_9L_A_10_9R,D_8R_9L_A_8L_9R,D_8R_9L_A_8R_9L,D_8R_9R_A_10_8L_9R,D_9L_A_9R,other
0,2020-11-01 01:30:00,D_8R_9L_A_10_8L_9R,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,2020-11-01 02:00:00,D_8R_9L_A_10_8L_9R,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,2020-11-01 02:30:00,D_8R_9L_A_10_8L_9R,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,2020-11-01 03:00:00,D_8R_9L_A_10_8L_9R,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,2020-11-01 03:30:00,D_8R_9L_A_10_8L_9R,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17512,2021-10-31 21:30:00,D_26L_27R_A_26R_27L_28,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17513,2021-10-31 22:00:00,D_26L_27R_A_26R_27L_28,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17514,2021-10-31 22:30:00,D_26L_27R_A_26R_27L_28,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17515,2021-10-31 23:00:00,D_26L_27R_A_26R_27L_28,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# 제너레이터 

## 사전설정

In [33]:
#기준 timestamp에서 과거 6시간 동안의 데이터를 호출 함 
def past_censor_data(
    airport_config_df: pd.DataFrame, std_timestamp: pd.Timestamp) -> Tuple[str, pd.DataFrame]:
    mask = airport_config_df["timestamp"] < std_timestamp
    subset = airport_config_df[mask][-12:]
    subset = np.array(subset.drop(columns = ['timestamp','airport_config']))
    return subset

#기준 timestamp에서 미래 6시간 동안의 데이터를 호출 함 
def future_censor_data(airport_config_df: pd.DataFrame, std_timestamp: pd.Timestamp):
    mask = airport_config_df["timestamp"] > std_timestamp
    subset = airport_config_df[mask][:12]
    subset = np.array(subset.drop(columns = ['timestamp','airport_config']))
    return subset

#기준 timestamps (input-output 연결 키 )
timestamps = list(airport_config['timestamp'])[12:-12]

In [19]:
def batch_censor_data(airport_config,batch_timestamps,mode):
  return_config = np.arange(len(batch_timestamps)*12*27).reshape(-1,12,27)
  if mode =='past':    
    for n,batch_timestamp in enumerate(batch_timestamps):
      temp_config = past_censor_data(airport_config,batch_timestamp)
      return_config[n,:,:] = temp_config
    return return_config
  if mode == 'future':
    for n,batch_timestamp in enumerate(batch_timestamps):
      temp_config = future_censor_data(airport_config,batch_timestamp)
      return_config[n,:,:] = temp_config
    return return_config

## 제너레이터 

In [63]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import Sequence
import math


class DataGenerator(Sequence):
  def __init__(self,
                 airport_config,
                 timestamps,
                 batch_size: int):
        self.airport_config = airport_config
        self.timestamps = timestamps
        self.batch_size = batch_size
        
  def __len__(self):
    return math.ceil(len(self.timestamps) / self.batch_size)
    
  def __getitem__(self, index):
    self.batch_timestamps = self.timestamps[index*self.batch_size:(index+1)*self.batch_size]
    #past data 
    past_configs = batch_censor_data(self.airport_config,self.batch_timestamps,'past')

    #future data 
    future_configs = batch_censor_data(self.airport_config,self.batch_timestamps,'future')

    return np.array(past_configs), np.array(future_configs)

#모델
**softmax 방식**
- 과거 6시간의 airport configuration 정보를  이용해 미래의 30분 이후 ~ 6시간 이후를 예측 
- ex : 11시에 예측을 시행한다 할 경우 11시 이전의 6시간(5~11시)까지의 데이터를 이용해 11시30분, 12시 ~ 17시 까지의 데이터들을 예측 
- 예측 하는 방식은 one hot encoding 형태의 configuration을 softmax로 확률 예측 
- 분류에선 softmax score로 분류를 했지만 여기서는 softmax score를 확률로써 그대로 사용 

**Regression 방식** <- 우선 regression 으로 진행 
- 과거의 6시간 데이터를 이용해 미래의 6시간을 통으로 예측 
- input : 과거 6시간 데이터 -> output : 미래 6시간  
- regression을 통해 나온 결과들을 min,max scaling을 통해 sum 1로 맞춰줌 

##사전설정

In [87]:
import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.metrics import Recall
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Bidirectional, LSTM, BatchNormalization,Input,Dropout,Dense,Conv2D,Concatenate,Flatten,Conv1D,Conv2D
from tensorflow.keras.utils import plot_model
from keras.layers.pooling import GlobalAveragePooling2D,GlobalAveragePooling3D,AveragePooling3D

##콜백

## 모델

In [ ]:
#Env 데이터 파트 
Env_input = Input((294,9)) #<---- shape 
x = Bidirectional(LSTM(128,return_sequences= True))(Env_input) #bidirec
x = Bidirectional(LSTM(128))(x)
x = Flatten()(x)
x = BatchNormalization()(x)
x = Dense(64, activation ='relu')(x)
x = BatchNormalization()(x)
x = Dense(32, activation ='relu')(x)
x = BatchNormalization()(x)
Env_output = Dense(10 ,activation='linear')(x)


In [89]:
config_input = Input((12,27))
x = Bidirectional(LSTM(128,return_sequences= True))(config_input) #bidirec
x = Bidirectional(LSTM(128,return_sequences= True))(x)
x = Conv1D(32,3,padding = 'same')(x)
out = Dense(27,activation='sigmoid')(x)

model = Model(config_input,out)
model.compile(optimizer='adam',loss = 'mse',metrics=['mae'])

In [90]:
model.summary()

Model: "model_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_21 (InputLayer)       [(None, 12, 27)]          0         
                                                                 
 bidirectional_13 (Bidirecti  (None, 12, 256)          159744    
 onal)                                                           
                                                                 
 bidirectional_14 (Bidirecti  (None, 12, 256)          394240    
 onal)                                                           
                                                                 
 conv1d (Conv1D)             (None, 12, 32)            24608     
                                                                 
 dense_16 (Dense)            (None, 12, 27)            891       
                                                                 
Total params: 579,483
Trainable params: 579,483
Non-traina

In [91]:
BATCH_SIZE=128
train_generator = DataGenerator(airport_config,timestamps,BATCH_SIZE)
in_con,out_con = next(iter(train_generator))

In [66]:
in_con.shape

(128, 12, 27)

In [69]:
out_con.shape

(128, 12, 27)

In [ ]:
history = model.fit(train_generator,epochs=25,verbose=1)

Epoch 1/25
137/137 [==============================] - 103s 688ms/step - loss: 0.0356 - mae: 0.0692
Epoch 2/25
137/137 [==============================] - 90s 654ms/step - loss: 0.0157 - mae: 0.0265
Epoch 3/25
137/137 [==============================] - 91s 662ms/step - loss: 0.0154 - mae: 0.0260
Epoch 4/25
137/137 [==============================] - 93s 679ms/step - loss: 0.0151 - mae: 0.0259
Epoch 5/25
137/137 [==============================] - 95s 694ms/step - loss: 0.0146 - mae: 0.0253
Epoch 6/25
129/137 [===========================>..] - ETA: 5s - loss: 0.0144 - mae: 0.0254

## Compile

##Cross Validation

# 예측 및 평가